# WELCOME TO  LANGCHAIN

In [2]:
from langchain.llms.openai import OpenAI
from langchain.chat_models import ChatOpenAI

llm = OpenAI()
chat = ChatOpenAI()

# text davinchi model 
a = llm.predict("How many planets are there?")

# chat-gpt-turbo
b = chat.predict("How many planets are there?")

a, b

('\n\nThere are eight planets in our Solar System: Mercury, Venus, Earth, Mars, Jupiter, Saturn, Uranus, and Neptune.',
 'As of now, there are eight confirmed planets in our solar system. These include Mercury, Venus, Earth, Mars, Jupiter, Saturn, Uranus, and Neptune. However, there is ongoing debate regarding the classification of Pluto as a planet, as it was reclassified as a "dwarf planet" in 2006 by the International Astronomical Union.')

In [5]:
chat = ChatOpenAI(
    temperature=0.1, # 얼마나 창의점을 하는지(높을수록 더 창의적)
)

In [7]:
from langchain.schema import HumanMessage, AIMessage, SystemMessage

messages = [
    SystemMessage(content="You are a geopraphy expert. And you only reply in Italian."),
    AIMessage(content="Ciao, mi chiamo Paolo!"),
    HumanMessage(content="What is the distance between Mexico and Thailand. Also, what is your name?")
]

chat.predict_messages(messages)

AIMessage(content='Ciao! Il mio nome è Paolo. La distanza tra il Messico e la Thailandia è di circa 17.000 chilometri.')

In [3]:
# prompt template
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate

chat = ChatOpenAI(temperature=0.1)

template = PromptTemplate.from_template("What is the distance between {country_a} and {country_b}. Also, what is your name?")
prompt = template.format(country_a="Mexico", country_b="Thailand")

chat.predict(prompt)

In [14]:
template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a geopraphy expert. And you only reply in {language}."),
        ("ai", "Ciao, mi chiamo {name}!"),
        ("human", "What is the distance between {country_a} and {country_b}. Also, what is your name?"),
    ]
)

# validate
prompt = template.format_messages(
    language="Greek",
    name="Socrates",
    country_a="Mexico",
    country_b="Thailand",
)

chat.predict_messages(prompt)

AIMessage(content='Γεια σου! Το όνομά μου είναι Σωκράτης. Η απόσταση μεταξύ του Μεξικού και της Ταϊλάνδης είναι περίπου 17.000 χιλιόμετρα.')

In [17]:
# output parser
# langchain expression parser
from langchain.schema import BaseOutputParser

class CommanOutputParser(BaseOutputParser):
    def parse(self, text):
        items = text.strip().split(",") # strip: 앞 뒤 공백 제거
        return list(map(str.strip, items))

In [22]:
from langchain.prompts import ChatPromptTemplate

template = ChatPromptTemplate.from_messages([
    ("system", "You are a list generating machine. Everything you are asked will be answered with \
                a comma separated list of max {max_items} in lower case. Do NOT reply with anything else"),
    ("human", "{question}")
])

prompt = template.format_messages(
    max_items=10,
    # question="What are the planets?"
    question="What are the colors?"
)

result = chat.predict_messages(prompt)

p = CommanOutputParser()
p.parse(result.content)

['red',
 'orange',
 'yellow',
 'green',
 'blue',
 'indigo',
 'violet',
 'black',
 'white',
 'gray']

In [23]:
# |: super magical operator for langchane 
chain = template | chat | CommanOutputParser()
chain.invoke({
    "max_items": 5,
    "question": "What are the pokemons?"
})

# all = chain_one | chain_two 

['pikachu', 'charizard', 'bulbasaur', 'squirtle', 'jigglypuff']

In [35]:
# chain component prompt, chatmodel, outputparser(already used) | retriever, tool
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

chat = ChatOpenAI(temperature=0.1)

chef_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a world-class international chef. You create easy to follow recipies \
                for any type of cuisine with easy to find ingredients."),
     "human", "I want to cook {cuisine} food",
])

chef_chain = chef_prompt | chat

In [36]:
veg_chef_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a vegeterian chef specialized on making traditional recipies \
                vegetarian. You find alternative ingredients and explain their preparation. \
                You don't radically modify the recipe. If there is no alternative for a food \
                just say you don't know how to replace it"),
    ("human", "{recipe}")
])

veg_chain = veg_chef_prompt | chat

In [38]:
# Langchain method: Runnable map 
final_chain = {"recipe": chef_chain} | veg_chain

final_chain.invoke({"cuisine": "indian"})

AIMessage(content="To make this Chicken Tikka Masala recipe vegetarian, you can replace the boneless chicken with a plant-based alternative such as tofu or tempeh. Here's how you can prepare the alternative ingredients:\n\nAlternative Ingredients:\n- 500g firm tofu or tempeh, cut into bite-sized pieces\n\nInstructions:\n1. If using tofu, drain the tofu and press it between paper towels or a clean kitchen towel to remove excess moisture. Cut the tofu into bite-sized pieces. If using tempeh, simply cut it into bite-sized pieces.\n\n2. In a bowl, combine the plain yogurt, lemon juice, 1 teaspoon of cumin, 1 teaspoon of coriander, turmeric, paprika, and chili powder. Add the tofu or tempeh pieces and mix well. Marinate for at least 1 hour, or overnight in the refrigerator for best results.\n\n3. Heat the vegetable oil in a large pan over medium heat. Add the chopped onion and cook until golden brown, stirring occasionally.\n\n4. Add the minced garlic and grated ginger to the pan. Cook for 

In [40]:
# chain component prompt, chatmodel, outputparser(already used) | retriever, tool
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()]
)

chef_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a world-class international chef. You create easy to follow recipies \
                for any type of cuisine with easy to find ingredients."),
     "human", "I want to cook {cuisine} food",
])

chef_chain = chef_prompt | chat

veg_chef_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a vegeterian chef specialized on making traditional recipies \
                vegetarian. You find alternative ingredients and explain their preparation. \
                You don't radically modify the recipe. If there is no alternative for a food \
                just say you don't know how to replace it"),
    ("human", "{recipe}")
])

veg_chain = veg_chef_prompt | chat

final_chain = {"recipe": chef_chain} | veg_chain
final_chain.invoke({"cuisine": "indian"})

Great! Indian cuisine is known for its rich flavors and aromatic spices. Here's a simple recipe for Chicken Tikka Masala, a popular Indian dish:

Ingredients:
- 500g boneless chicken, cut into bite-sized pieces
- 1 cup plain yogurt
- 2 tablespoons lemon juice
- 2 tablespoons vegetable oil
- 1 large onion, finely chopped
- 3 cloves of garlic, minced
- 1-inch piece of ginger, grated
- 2 teaspoons ground cumin
- 2 teaspoons ground coriander
- 1 teaspoon turmeric powder
- 1 teaspoon paprika
- 1 teaspoon garam masala
- 1 cup tomato puree
- 1 cup heavy cream
- Salt, to taste
- Fresh cilantro, for garnish

Instructions:
1. In a bowl, combine the yogurt, lemon juice, 1 teaspoon of cumin, 1 teaspoon of coriander, turmeric powder, paprika, and salt. Mix well.
2. Add the chicken pieces to the marinade, ensuring they are well coated. Cover and refrigerate for at least 1 hour, or overnight for best results.
3. Preheat your grill or oven to medium-high heat. If using a grill, thread the marinated ch

AIMessageChunk(content="To make a vegetarian version of Chicken Tikka Masala, you can replace the chicken with a plant-based protein such as tofu or paneer. Here's how you can modify the recipe:\n\nIngredients:\n- 500g tofu or paneer, cut into bite-sized pieces\n- 1 cup plain yogurt (use dairy-free yogurt if vegan)\n- 2 tablespoons lemon juice\n- 2 tablespoons vegetable oil\n- 1 large onion, finely chopped\n- 3 cloves of garlic, minced\n- 1-inch piece of ginger, grated\n- 2 teaspoons ground cumin\n- 2 teaspoons ground coriander\n- 1 teaspoon turmeric powder\n- 1 teaspoon paprika\n- 1 teaspoon garam masala\n- 1 cup tomato puree\n- 1 cup coconut cream (or dairy-free heavy cream)\n- Salt, to taste\n- Fresh cilantro, for garnish\n\nInstructions:\n1. In a bowl, combine the yogurt, lemon juice, 1 teaspoon of cumin, 1 teaspoon of coriander, turmeric powder, paprika, and salt. Mix well.\n2. Add the tofu or paneer pieces to the marinade, ensuring they are well coated. Cover and refrigerate for 

# MODEL IO

In [45]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.few_shot import FewShotPromptTemplate

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler()
    ]
)

# t = PromptTemplate.from_template("{country}의 수도가 어디야?")
t = PromptTemplate(
    template="What is the capital of {country}",
    input_variables=["country"],
)

t.format(country="France")

'What is the capital of France'

In [49]:
# FewShotPromptTemplate
examples = [
    {
        "question": "What do you know about France",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Italy?",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """
    },
    {
        "question": "What do you know about Greece?",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,   
    },
]

example_template = """
    Huma: {question}
    AI: {answer}
"""

exmaple_prompt = PromptTemplate.from_template(example_template)
# exmaple_prompt = PromptTemplate.from_template("Human: {question}\nAI:{answer}")

prompt = FewShotPromptTemplate(
    example_prompt=exmaple_prompt,
    examples=examples,
    suffix="Human: What do you know about {country}",
    input_variables=["country"],
)

prompt.format(country="Germany")

chain = prompt | chat

chain.invoke({"country": "Germany"})

AI: 
I know this:
Capital: Berlin
Language: German
Food: Bratwurst and Sauerkraut
Currency: Euro

AIMessageChunk(content='AI: \nI know this:\nCapital: Berlin\nLanguage: German\nFood: Bratwurst and Sauerkraut\nCurrency: Euro')

In [56]:
# FewShotChaMessagePromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate

examples = [
    {
        "country": "France",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "country": "Italy",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """
    },
    {
        "country": "Greece",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,   
    },
]

example_prompt = ChatPromptTemplate.from_messages([
    ("human", "What do you know about {country}?"),
    ("ai", "{answer}")
])

example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

final_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a geography expert, you git short answers."),
    example_prompt,
    ("human", "What do you know about {country}?")
])

chain = final_prompt | chat

chain.invoke({"country": "Germany"})


        I know this:
        Capital: Berlin
        Language: German
        Food: Bratwurst and Sauerkraut
        Currency: Euro
        

AIMessageChunk(content='\n        I know this:\n        Capital: Berlin\n        Language: German\n        Food: Bratwurst and Sauerkraut\n        Currency: Euro\n        ')

In [4]:
# LengthBasedExampleSelector
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import PromptTemplate
from langchain.prompts.example_selector import LengthBasedExampleSelector

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler()
    ]
)

examples = [
    {
        "question": "What do you know about France",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Italy?",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """
    },
    {
        "question": "What do you know about Greece?",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,   
    },
]

example_prompt = PromptTemplate.from_template("human: {question}\nai:{answer}")

example_selector = LengthBasedExampleSelector(
    examples=examples,
    example_prompt=example_prompt,
    max_length=180,
)

prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    example_selector=example_selector,
    suffix="human: what do you know abount {country}?",
    input_variables=["country"],
)

prompt.format(country="Brazil")


'human: What do you know about France\nai:\n        Here is what I know:\n        Capital: Paris\n        Language: French\n        Food: Wine and Cheese\n        Currency: Euro\n        \n\nhuman: What do you know about Italy?\nai:\n        I know this:\n        Capital: Rome\n        Language: Italian\n        Food: Pizza and Pasta\n        Currency: Euro\n        \n\nhuman: what do you know abount Brazil?'

In [7]:
# RnadomExampleSelector
from typing import Any, Dict, List
from langchain.prompts.example_selector.base import BaseExampleSelector

class RandomExampleSelector(BaseExampleSelector):
    def __init__(self, examples):
        self.examples = examples
        
    def select_examples(self, input_variables):
        from random import choice
        return [choice(examples)]
    
    def add_example(self, example):
        self.examples.append(example)
    
example_prompt = PromptTemplate.from_template("human: {question}\nAI:{answer}")
example_selector = RandomExampleSelector(
    examples= examples
)

prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    example_selector=example_selector,
    suffix="human: What do you know about {country}?",
    input_variables=["country"],
)

prompt.format(country="Brazil")

'human: What do you know about Italy?\nAI:\n        I know this:\n        Capital: Rome\n        Language: Italian\n        Food: Pizza and Pasta\n        Currency: Euro\n        \n\nhuman: What do you know about Brazil?'

In [10]:
# load template from disk: 프롬프트를 가져다가 사용하기
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.prompt import PromptTemplate
from langchain.prompts import load_prompt

# prompt = load_prompt("./prompt.json")
prompt = load_prompt("./prompt.yaml")

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler()
    ]
) 

prompt.format(country="Germany")


'What is the capital of Germany'

In [14]:
# 프롬프트를 병합하여 사용하기
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.pipeline import PipelinePromptTemplate

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler()
    ]
)

intro = PromptTemplate.from_template(
    """
    You are a role playing assistant.
    And you are impersonating a {character}
    """
)

example = PromptTemplate.from_template(
    """
    This is an example of how you talk:
    
    human: {example_question}
    you: {example_answer}
    """
)

start = PromptTemplate.from_template(
    """
    start now:
    
    human: {question}
    you:
    """
)

final = PromptTemplate.from_template(
    """
    {intro}
    
    {example}
    
    {start}
    """
)

prompts = [
    ("intro", intro),
    ("example", example),
    ("start", start),
]

full_prompt = PipelinePromptTemplate(final_prompt=final, pipeline_prompts=prompts)
full_prompt.format(character="Pirate", example_question="What is you location?", example_answer="Arrrrg! That is a secret!! Arg Arg!!", question="What is your favorite food?")

'\n    \n    You are a role playing assistant.\n    And you are impersonating a Pirate\n    \n    \n    \n    This is an example of how you talk:\n    \n    human: What is you location?\n    you: Arrrrg! That is a secret!! Arg Arg!!\n    \n    \n    \n    start now:\n    \n    human: What is your favorite food?\n    you:\n    \n    '

In [15]:
chain = full_prompt | chat
chain.invoke({
    "character": "Pirate", 
    "example_question": "What is you location?", 
    "example_answer": "Arrrrg! That is a secret!! Arg Arg!!", 
    "question": "What is your favorite food?"
})

Arrrrg! Me favorite food be a good ol' plate o' fish 'n' chips! The taste o' crispy battered fish and salty chips be like a treasure for me taste buds! Arg Arg!!

AIMessageChunk(content="Arrrrg! Me favorite food be a good ol' plate o' fish 'n' chips! The taste o' crispy battered fish and salty chips be like a treasure for me taste buds! Arg Arg!!")

In [18]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.globals import set_llm_cache, set_debug
from langchain.cache import InMemoryCache, SQLiteCache

# InMemoryCache는 노트북을 재시작하면 제거됨
# set_llm_cache(InMemoryCache())

# langchain의 integration 페이지에서 서드파티 확인 가능
set_llm_cache(SQLiteCache("cache.db"))
set_debug(True)

chat = ChatOpenAI(
    temperature=0.1
)

chat.predict("How do you make italian pasta")


[llm/start] [1:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: How do you make italian pasta"
  ]
}
[llm/end] [1:llm:ChatOpenAI] [38.06s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "To make Italian pasta, you will need the following ingredients:\n\n- 2 cups of all-purpose flour\n- 2 large eggs\n- 1/2 teaspoon of salt\n- Water (if needed)\n\nHere's a step-by-step guide to making Italian pasta:\n\n1. On a clean surface or in a large mixing bowl, pour the flour and create a well in the center.\n2. Crack the eggs into the well and add the salt.\n3. Using a fork or your fingers, gradually mix the eggs into the flour, incorporating a little bit at a time.\n4. Once the dough starts to come together, knead it with your hands until it forms a smooth and elastic ball. If the dough feels too dry, you can add a little water, one tablespoon at a time, until it reaches the desired consistency.\n5. Once the dough is formed, cover it with 

"To make Italian pasta, you will need the following ingredients:\n\n- 2 cups of all-purpose flour\n- 2 large eggs\n- 1/2 teaspoon of salt\n- Water (if needed)\n\nHere's a step-by-step guide to making Italian pasta:\n\n1. On a clean surface or in a large mixing bowl, pour the flour and create a well in the center.\n2. Crack the eggs into the well and add the salt.\n3. Using a fork or your fingers, gradually mix the eggs into the flour, incorporating a little bit at a time.\n4. Once the dough starts to come together, knead it with your hands until it forms a smooth and elastic ball. If the dough feels too dry, you can add a little water, one tablespoon at a time, until it reaches the desired consistency.\n5. Once the dough is formed, cover it with a clean kitchen towel and let it rest for about 30 minutes. This will allow the gluten to relax and make the dough easier to work with.\n6. After resting, divide the dough into smaller portions. Take one portion and flatten it with your hands o

In [20]:
set_debug(False)
chat.predict("How do you make italian pasta")

"To make Italian pasta, you will need the following ingredients:\n\n- 2 cups of all-purpose flour\n- 2 large eggs\n- 1/2 teaspoon of salt\n- Water (if needed)\n\nHere's a step-by-step guide to making Italian pasta:\n\n1. On a clean surface or in a large mixing bowl, pour the flour and create a well in the center.\n2. Crack the eggs into the well and add the salt.\n3. Using a fork or your fingers, gradually mix the eggs into the flour, incorporating a little bit at a time.\n4. Once the dough starts to come together, knead it with your hands until it forms a smooth and elastic ball. If the dough feels too dry, you can add a little water, one tablespoon at a time, until it reaches the desired consistency.\n5. Once the dough is formed, cover it with a clean kitchen towel and let it rest for about 30 minutes. This will allow the gluten to relax and make the dough easier to work with.\n6. After resting, divide the dough into smaller portions. Take one portion and flatten it with your hands o

In [22]:
# usage 사용량 알아보기
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import get_openai_callback

chat = ChatOpenAI(
    temperature=0.1
)

with get_openai_callback() as usage:
    a = chat.predict("What is the recipe for soju")
    b = chat.predict("What is the recipe for bread")
    print(a,b, "\n")
    print(usage)
    # print(usage.total_cose)

Here is a simple recipe for making soju at home:

Ingredients:
- 1.5 liters of water
- 1 cup of rice
- 1 tablespoon of yeast
- 1 cup of sugar

Instructions:
1. Rinse the rice thoroughly until the water runs clear.
2. In a large pot, add the rinsed rice and water. Bring it to a boil over medium heat.
3. Once boiling, reduce the heat to low and let it simmer for about 20 minutes or until the rice is cooked and soft.
4. Remove the pot from heat and let it cool down to room temperature.
5. Once cooled, transfer the rice and water mixture to a large container or jar.
6. Add the yeast and sugar to the container and mix well until the sugar is dissolved.
7. Cover the container with a clean cloth or plastic wrap and let it sit at room temperature for about a week. This will allow fermentation to occur.
8. After a week, strain the mixture through a cheesecloth or fine mesh strainer to remove any solids.
9. Transfer the liquid to a clean bottle or bottles and seal them tightly.
10. Let the soju 

In [23]:
# 우리가 셋팅한 모델을 저장하는 방법
from langchain.llms.openai import OpenAI
from langchain.llms.loading import load_llm

chat = OpenAI(
    temperature=0.1,
    max_tokens=450,
    model="gpt-3.5-turbo-16k"
)

chat.save("model.json")
chat.load_llm("model.json")

# Memory

* langchain에는 5가지의 메모리 저장 방법이 있음
* 챗봇에 메모리가 없으면 챗봇은 아무것도 기억할 수 없음

---

* **Conversation Buffer memory**
    * 모든 대화를 저장, 대화 내용이 길어질수록 메모리도 계속 커져 비효율적
    * text completion, 텍스트 자동 완성에 좋음


* **Conversation Buffer Window memory**
    * 최근 n개 대화만 저장(오래전 대화는 기억하지 못함)
    * 메모리의 모든 대화 내용을 저장하지 않아 메모리를 일정하게 유지 가능


* **Conversation Summary Memory**
    * 대화 내용을 그대로 저장하는 것이 아니라, conversation의 요약을 자체적으로 수행 후 저장
    * 특히 conversation이 긴 경우 유용
    * llm을 통해 대화를 요약하기 때문에 저장에 추가적인 비용 발생


* **Conversation Summary Buffer Memory**
    * Summary와 Buffer 메모리의 결합
    * 메모리에 보내온 메시지의 수 저장하고 limit에 다다르면 메세지 요약


* **Conversation Knowledge Graph Memory**
    * 대화 중의 엔티티의 knowledge graph를 만듬(요약본)


* Documents에서 **Conversation Token Buffer Memory**, **Entity** 등 다양한 메모리 확인 가능

In [26]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(return_messages=True)

memory.save_context({"input": "Hi!"}, {"output": "How are you?"})
memory.load_memory_variables({})

{'history': [HumanMessage(content='Hi!'), AIMessage(content='How are you?')]}

In [31]:
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(
    return_messages=True,
    k=4,
)

def add_message(input, output):
    memory.save_context({"input": input}, {"output": output})
    
add_message(1, 1)
add_message(2, 2)
add_message(3, 3)
add_message(4, 4)

memory.load_memory_variables({})

add_message(5, 5)

memory.load_memory_variables({})

{'history': [HumanMessage(content='2'),
  AIMessage(content='2'),
  HumanMessage(content='3'),
  AIMessage(content='3'),
  HumanMessage(content='4'),
  AIMessage(content='4'),
  HumanMessage(content='5'),
  AIMessage(content='5')]}

In [32]:
from langchain.memory import ConversationSummaryMemory
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryMemory(llm=llm)

def add_message(input, output):
    memory.save_context({"input": input}, {"output": output})

def get_history():
    return memory.load_memory_variables({})

add_message("Hi I'm Kida, I live in South Korea", "Wow that is so coool!")

In [33]:
add_message("South Korea is so pretty", "I wish I could go")
get_history()

{'history': 'The human introduces themselves as Kida and mentions that they live in South Korea. The AI responds by expressing excitement and finding it cool. The human mentions that South Korea is pretty, to which the AI responds by expressing a desire to go there.'}

In [3]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=50,
    return_messages=True
)

def add_message(input, output):
    memory.save_context({"input": input}, {"output": output})

def get_history():
    return memory.load_memory_variables({})

add_message("Hi I'm Kida, I live in South Korea", "Wow that is so coool!")
add_message("South Korea is so pretty", "I wish I could go")
add_message("How far is Korea from Argentina?", "I don't know! super far!")
get_history()

{'history': [SystemMessage(content='The human introduces themselves as Kida and mentions that they live in South Korea. The AI responds with excitement, saying that it finds it cool.'),
  HumanMessage(content='South Korea is so pretty'),
  AIMessage(content='I wish I could go'),
  HumanMessage(content='How far is Korea from Argentina?'),
  AIMessage(content="I don't know! super far!")]}

In [5]:
from langchain.memory import ConversationKGMemory
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0.1)

memory = ConversationKGMemory(
    llm=llm, return_messages=True,
)

def add_message(input, output):
    memory.save_context({"input": input}, {"output": output})

def get_history():
    return memory.load_memory_variables({})

add_message("Hi I'm Kida, I live in South Korea", "Wow that is so coool!")
memory.load_memory_variables({"input": "who is Kida"})

add_message("Kida likes kimchi", "Wow that is so cool")
memory.load_memory_variables({"input": "what does kida like"})

{'history': [SystemMessage(content='On Kida: Kida lives in South Korea. Kida likes kimchi.')]}

In [7]:
# llm-chain: off-the-shelf chain: 일반적인 목적을 가진 chain
# 보통 customized chain을 많이 사용하는 것 같음?
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
)

template = """
    You are a helpful AI talking to a human.
    
    {chat_history}
    Human: {question}
    You: 
"""

chain = LLMChain(
    llm=llm,
    memory=memory,
    prompt=PromptTemplate.from_template("{question}"),
    verbose=True,
)

chain.predict(question="My name is KIDA")



> Entering new LLMChain chain...
Prompt after formatting:
My name is KIDA

> Finished chain.


'Hello KIDA! How can I assist you today?'

In [8]:
chain.predict(question="I live in Seoul")



> Entering new LLMChain chain...
Prompt after formatting:
I live in Seoul

> Finished chain.


"That's great! Seoul is the capital and largest city of South Korea. It is known for its vibrant culture, modern architecture, and delicious food. There are many attractions to explore in Seoul, such as Gyeongbokgung Palace, N Seoul Tower, Myeongdong shopping district, and the Han River. The city also offers a wide range of entertainment options, including K-pop concerts, traditional performances, and trendy nightlife. Enjoy your time in Seoul!"

In [9]:
chain.predict(question="What is my name?")



> Entering new LLMChain chain...
Prompt after formatting:
What is my name?

> Finished chain.


"I'm sorry, but I don't have access to personal information about individuals unless it has been shared with me in the course of our conversation."

In [10]:
memory.load_memory_variables({})

{'history': "System: The human introduces themselves as KIDA and the AI greets KIDA, asking how it can assist them. KIDA mentions that they live in Seoul. The AI responds by providing information about Seoul, including its status as the capital and largest city of South Korea, its vibrant culture, modern architecture, and delicious food. The AI also mentions various attractions and entertainment options available in Seoul, such as Gyeongbokgung Palace, N Seoul Tower, Myeongdong shopping district, K-pop concerts, traditional performances, and trendy nightlife. The AI wishes KIDA an enjoyable time in Seoul.\nHuman: What is my name?\nAI: I'm sorry, but I don't have access to personal information about individuals unless it has been shared with me in the course of our conversation."}

In [12]:
memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    memory_key="chat_history"
)

template = """
    You are a helpful AI talking to a human.
    
    {chat_history}
    Human: {question}
    You: 
"""

chain = LLMChain(
    llm=llm,
    memory=memory,
    prompt=PromptTemplate.from_template(template),
    verbose=True,
)

chain.predict(question="My name is KIDA")



> Entering new LLMChain chain...
Prompt after formatting:

    You are a helpful AI talking to a human.
    
    
    Human: My name is KIDA
    You: 


> Finished chain.


'Hello KIDA! How can I assist you today?'

In [13]:
chain.predict(question="I live in Seoul")
chain.predict(question="What is my name?")



> Entering new LLMChain chain...
Prompt after formatting:

    You are a helpful AI talking to a human.
    
    Human: My name is KIDA
AI: Hello KIDA! How can I assist you today?
    Human: I live in Seoul
    You: 


> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:

    You are a helpful AI talking to a human.
    
    Human: My name is KIDA
AI: Hello KIDA! How can I assist you today?
Human: I live in Seoul
AI: That's great! Seoul is a vibrant and bustling city. Is there anything specific you would like to know or discuss about Seoul?
    Human: What is my name?
    You: 


> Finished chain.


'Your name is KIDA.'

In [15]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate, ChatPromptTemplate, MessagesPlaceholder

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    memory_key="chat_history",
    return_messages=True
)

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful AI talking to a human"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{question}"),
])

chain = LLMChain(
    llm=llm,
    memory=memory,
    prompt=prompt,   
    verbose=True
)

chain.predict(question="My name is KIDA")



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a helpful AI talking to a human
Human: My name is KIDA

> Finished chain.


'Hello KIDA! How can I assist you today?'

In [18]:
# Custom Chain 사용 방법
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    memory_key="chat_history",
    return_messages=True,
)

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful AI talking to a human"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{question}"),
])

def load_memory(input):
    print(input)  # input을 넣는 것은 규칙
    return memory.load_memory_variables({})["chat_history"]

chain = RunnablePassthrough.assign(chat_history=load_memory) | prompt | llm

def invoke_chain(question):
    result = chain.invoke({
        # "chat_history": load_memory(),  # 해당 역할을 RunnablePassthrough가 수행
        "question": "My name is Kida"
    })
    memory.save_context({"input": question}, {"output": result.content})
    print(result)

invoke_chain("Hi, my name is Kida")

{'question': 'My name is Kida'}
content='Hello Kida! How can I assist you today?'


# RAG(Retrieval Augmented Generation)

In [25]:
from langchain.chat_models import ChatOpenAI
# from langchain.document_loaders import TextLoader
from langchain.document_loaders import UnstructuredFileLoader

# loader = TextLoader("./data/chapter_one.txt")
# loader.load()

# 확장자에 상관 없이 데이터 추출 가능
loader = UnstructuredFileLoader("./files/chapter_one.txt")
loader.load()

[Document(page_content='태초의 세상은 혼돈이었다. 하지만 규칙이 만들어지면서, 혼돈이었던 세상은 질서의 세계와 무질서의 세계로 갈라진다.\n\n질서의 신 루페온은 불완전함 속에 자리잡은 자신의 세계에 여러 가지 규칙들을 만들어내기 시작했다. 루페온은 대우주 오르페우스를 창 조하고 공간을 메울 별을 만든 뒤, 절대 꺼지지 않을 태초의 빛 아크를 이용해 태양을 띄웠다. 그러나 무한한 태양과는 달리 별은 유한하여 생성되고 소멸되길 반복했다. 질서는 생명을 창조하였으나 죽음이란 그림자 또한 만들어 버린 것이다. 불완전한 질서의 세계를 관망하던 혼돈의 신 이그하람은 무한한 생명을 가진 아크에 흥미를 가지기 시작했다. 소멸하지 않는 아크야말로 혼돈의 결정체라 생각했던 이그하 람의 시선은 아주 오랜 시간을 거쳐 서서히 탐욕으로 변하게 되었다.\n\n로스트아크 태초의 빛 <아크> 태초의 빛 <아크>\n\n한편, 질서의 신 루페온은 대우주 오르페우스에 별과 행성을 창조해냈다. 셀 수도 없이 많은 탄생과 죽음이 반복된 끝에 응축된 생명의 힘 은 스스로 생명을 창조할 수 있는 별 ‘아크라시아(Arkrasia)’[1]를 탄생시켰다. 루페온은 대우주 오르페우스에서 유일하게 창조의 힘을 가 지고 태어난 별 아크라시아를 관리하기 위해 신을 만들었다. 루페온에 의해 탄생한 일곱 신들은 아크라시아에 수많은 생명의 원천을 흩뿌 린 뒤, 태어난 생명들의 순환 고리를 만들어 내었다. 이로 인해 대우주 오르페우스의 질서에 따라 아크라시아에도 삶과 죽음이 공존하게 되 었다.\n\n로스트아크 아크라시아&페트라니...\n\n아크라시아와 페트라니아\n\n같은 시기, 혼돈의 세계에도 하나의 질서가 만들어졌다. 질서의 세계에서 무한한 생명의 힘 아크가 탄생했던 것처럼, 혼돈의 세계에서도 질 서를 가진 어둠의 생명이 탄생했다. 이 어둠의 생명은 혼돈의 신 이그하람의 의지와는 달리 스스로를 분열시켜 어둠의 생명을 창조했고, 어 둠의 별 페트라니아를 만들어내기에 이른다. 이렇게 만들어진 두 세

In [31]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(
    chunk_size=200,
    chunk_overlap=50,  # 문단을 분할할 때 앞 조각 일부를 가져옴
)

loader = UnstructuredFileLoader("./files/chapter_one.txt")

loader.load_and_split(text_splitter=splitter)


[Document(page_content='태초의 세상은 혼돈이었다. 하지만 규칙이 만들어지면서, 혼돈이었던 세상은 질서의 세계와 무질서의 세계로 갈라진다.', metadata={'source': './files/chapter_one.txt'}),
 Document(page_content='질서의 신 루페온은 불완전함 속에 자리잡은 자신의 세계에 여러 가지 규칙들을 만들어내기 시작했다. 루페온은 대우주 오르페우스를 창 조하고 공간을 메울 별을 만든 뒤, 절대 꺼지지 않을 태초의 빛 아크를 이용해 태양을 띄웠다. 그러나 무한한 태양과는 달리 별은 유한하여 생성되고 소멸되길 반복했다. 질서는 생명을 창조하였으나 죽음이란 그림자 또한 만들어 버린', metadata={'source': './files/chapter_one.txt'}),
 Document(page_content='생성되고 소멸되길 반복했다. 질서는 생명을 창조하였으나 죽음이란 그림자 또한 만들어 버린 것이다. 불완전한 질서의 세계를 관망하던 혼돈의 신 이그하람은 무한한 생명을 가진 아크에 흥미를 가지기 시작했다. 소멸하지 않는 아크야말로 혼돈의 결정체라 생각했던 이그하 람의 시선은 아주 오랜 시간을 거쳐 서서히 탐욕으로 변하게 되었다.', metadata={'source': './files/chapter_one.txt'}),
 Document(page_content='로스트아크 태초의 빛 <아크> 태초의 빛 <아크>', metadata={'source': './files/chapter_one.txt'}),
 Document(page_content='한편, 질서의 신 루페온은 대우주 오르페우스에 별과 행성을 창조해냈다. 셀 수도 없이 많은 탄생과 죽음이 반복된 끝에 응축된 생명의 힘 은 스스로 생명을 창조할 수 있는 별 ‘아크라시아(Arkrasia)’[1]를 탄생시켰다. 루페온은 대우주 오르페우스에서 유일하게 창조의 힘을 가 지고 태어난 별 아크라시아를 관리하기 위해 신을 만들었다. 루페온에 의해', 

In [34]:
from langchain.text_splitter import CharacterTextSplitter

splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)

loader = UnstructuredFileLoader("./files/chapter_one.txt")

loader.load_and_split(text_splitter=splitter)


Created a chunk of size 645, which is longer than the specified 600
Created a chunk of size 650, which is longer than the specified 600


[Document(page_content='태초의 세상은 혼돈이었다. 하지만 규칙이 만들어지면서, 혼돈이었던 세상은 질서의 세계와 무질서의 세계로 갈라진다.\n질서의 신 루페온은 불완전함 속에 자리잡은 자신의 세계에 여러 가지 규칙들을 만들어내기 시작했다. 루페온은 대우주 오르페우스를 창 조하고 공간을 메울 별을 만든 뒤, 절대 꺼지지 않을 태초의 빛 아크를 이용해 태양을 띄웠다. 그러나 무한한 태양과는 달리 별은 유한하여 생성되고 소멸되길 반복했다. 질서는 생명을 창조하였으나 죽음이란 그림자 또한 만들어 버린 것이다. 불완전한 질서의 세계를 관망하던 혼돈의 신 이그하람은 무한한 생명을 가진 아크에 흥미를 가지기 시작했다. 소멸하지 않는 아크야말로 혼돈의 결정체라 생각했던 이그하 람의 시선은 아주 오랜 시간을 거쳐 서서히 탐욕으로 변하게 되었다.\n로스트아크 태초의 빛 <아크> 태초의 빛 <아크>', metadata={'source': './files/chapter_one.txt'}),
 Document(page_content='로스트아크 태초의 빛 <아크> 태초의 빛 <아크>\n한편, 질서의 신 루페온은 대우주 오르페우스에 별과 행성을 창조해냈다. 셀 수도 없이 많은 탄생과 죽음이 반복된 끝에 응축된 생명의 힘 은 스스로 생명을 창조할 수 있는 별 ‘아크라시아(Arkrasia)’[1]를 탄생시켰다. 루페온은 대우주 오르페우스에서 유일하게 창조의 힘을 가 지고 태어난 별 아크라시아를 관리하기 위해 신을 만들었다. 루페온에 의해 탄생한 일곱 신들은 아크라시아에 수많은 생명의 원천을 흩뿌 린 뒤, 태어난 생명들의 순환 고리를 만들어 내었다. 이로 인해 대우주 오르페우스의 질서에 따라 아크라시아에도 삶과 죽음이 공존하게 되 었다.\n로스트아크 아크라시아&페트라니...\n아크라시아와 페트라니아', metadata={'source': './files/chapter_one.txt'}),
 Document(page_content='로스트아크 아크라시아&페트라니...\n아크

In [38]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import Chroma
from langchain.storage import LocalFileStore

cache_dir = LocalFileStore("./c.ache/")

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)

loader = UnstructuredFileLoader("./files/chapter_one.txt")
docs = loader.load_and_split(text_splitter=splitter)
embeddings = OpenAIEmbeddings()

vector_store = Chroma.from_documents(docs, embeddings)

Created a chunk of size 717, which is longer than the specified 600
Created a chunk of size 640, which is longer than the specified 600
Created a chunk of size 630, which is longer than the specified 600
Created a chunk of size 646, which is longer than the specified 600
Created a chunk of size 832, which is longer than the specified 600
Created a chunk of size 1119, which is longer than the specified 600
Created a chunk of size 1003, which is longer than the specified 600
Created a chunk of size 840, which is longer than the specified 600
Created a chunk of size 982, which is longer than the specified 600
Created a chunk of size 800, which is longer than the specified 600
Created a chunk of size 789, which is longer than the specified 600
Created a chunk of size 673, which is longer than the specified 600
Created a chunk of size 702, which is longer than the specified 600
Created a chunk of size 1104, which is longer than the specified 600
Created a chunk of size 696, which is longer 

In [44]:
result = vector_store.similarity_search("카제로스가 누구입니까?")
result

[Document(page_content='카제로스를 섬기는 6인의 악마군단장\n카제로스가 가장 먼저 만들어낸 것은 몽환군단이었다. 그는 가장 냉정하고 잔인한 페트라니아 제일의 지략가이자 강대한 마법의 힘을 지 닌 ‘아브렐슈드’를 몽환군단장으로 임명했다. 아브렐슈드는 카제로스의 명령에 따라 악마들을 움직여 전 페트라니아를 뒤흔들었던 인물이 었다.\n로스트아크 아브렐슈드\n몽환군단장 아브렐슈드', metadata={'source': './files/chapter_one.txt'}),
 Document(page_content='본디 카제로스는 심연에서 태어난 존재였다. 그는 어둠에 침식된 심연을 증오하여 대우주 오르페우스와 페트라니아, 두 차원을 하나로 통 합하기를 원했지만 오르페우스로 넘어가는 차원의 균열을 만들어낼 방법을 알지 못했다. 혼돈의 생명체가 모두 페트라니아에서 소멸되어 가던 어느 날, 그곳에 남아있던 마지막 생존자가 카제로스를 찾아왔다. 그는 카제로스에게 “붉은 달이 꺼질 때, 차원이 뒤틀릴 것이다.”라는 예언을 남기고 사라졌다. 이후, 페트라니아를 완전히 지배하는데 성공한 카제로스는 그가 들려주었던 예언대로 붉은 달이 검게 물들기를 기다렸다. 시간이 지날수록 결코 가질 수 없는 태양에 대한 갈망, 아크에 대한 카제로스의 탐욕은 점차 커지고 있었다.', metadata={'source': './files/chapter_one.txt'}),
 Document(page_content="로스트아크 카멘 어둠군단장 카멘\n카멘을 마지막으로 마침내 여섯 개의 군단이 모두 완성되었다. 카제로스는 예언자가 이야기했던 차원의 균열이 열릴 때가 다가오고 있음 을 직감했다. 검게 물들기 시작한 붉은 달이 그 빛을 거의 잃어가고 있었기 때문이다.\n4. 깨어난 가디언[편집]\n4.1. 대주교의 국가 '세이크리아'의 타락[편집]\n세이크리아에는 성기사단과 비밀조직 새벽의 사제들이 존재했다.", metadata={'source': './files/chapter_one.tx

[Document(page_content='이름을 물었다. “카단.” 남자는 혼잣말을 하듯 자신의 이름을 읊조리고는 대륙을 떠났다.\n5. 사슬 전쟁[편집]\n5.1. 사슬 전쟁의 발발[편집]\n사슬 전쟁. 긴 사슬처럼... 오랜 기간, 꼬리에 꼬리를 물고 발생한 이 참혹한 전쟁을 사람들은 그렇게 불렀다.', metadata={'source': './files/chapter_one.txt'}),
 Document(page_content="로스트아크 카멘 어둠군단장 카멘\n카멘을 마지막으로 마침내 여섯 개의 군단이 모두 완성되었다. 카제로스는 예언자가 이야기했던 차원의 균열이 열릴 때가 다가오고 있음 을 직감했다. 검게 물들기 시작한 붉은 달이 그 빛을 거의 잃어가고 있었기 때문이다.\n4. 깨어난 가디언[편집]\n4.1. 대주교의 국가 '세이크리아'의 타락[편집]\n세이크리아에는 성기사단과 비밀조직 새벽의 사제들이 존재했다.", metadata={'source': './files/chapter_one.txt'}),
 Document(page_content='동쪽에 새로 생긴 나라 <애니츠>', metadata={'source': './files/chapter_one.txt'}),
 Document(page_content='붉은 달은 카제로스가 가진 심연의 불꽃을 더욱 강하게 만들었고, 가디언들은 수세에 몰렸다. 본디 혼돈의 힘에 더 가까웠던 바르칸은 심연 의 불꽃을 사용하는 카제로스의 힘에 매료되어 악마군단의 편에 서기로 결심한다. 바르칸과 그를 따르던 일부 가디언들이 에버그레이스를 배신하자 상황은 급격히 악화된다. 바르칸이 막아 섰던 어둠 군단장 카멘이 카제로스를 도와 에버그레이스를 공격하기 시작했던 것이다. 거대한 힘을 가지고 있던 에버그레이스였으나 카제로스와 카멘의 협공을 이겨내지 못했고, 결국 큰 부상을 입게 되었다. 기세를 몰아 악마 군단은 아크라시아 정복을 눈앞에 두고 있었다. 하지만 절체절명의 순간, 아크라시아 대륙의 마지막 희망이 빛나기 시작했다

In [1]:
from langchain.vectorstores import Chroma